In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv(r'C:/Users/uzmap/Documents/GitHub/ForEx/processed_data.csv',index_col='Unnamed: 0')
data.head()

,Date,Open_price,Day_high,Day_low,Closing_price,Currency Pair,Trend_Open_price,Seasonal_Open_price,Residual_Open_price,Trend_Closing_price,...,Seasonal_Day_low,Residual_Day_low,RSI,MACD,MACD_Signal,BB_Upper,BB_Lower,day_of_week,month,is_weekend
0,2014-11-07,0.418073,0.415411,0.420199,0.418141,USD/INR,0.430544,0.482937,0.45972,0.430893,...,0.532825,0.464686,0.644914,0.494863,0.468996,0.371949,0.429018,0.666667,0.909091,0.0
1,2014-11-10,0.418829,0.415513,0.420199,0.418794,USD/INR,0.430544,0.503235,0.45972,0.430893,...,0.520792,0.464686,0.644914,0.495321,0.469106,0.371949,0.429018,0.000000,0.909091,0.0
2,2014-11-11,0.419035,0.414969,0.421339,0.418883,USD/INR,0.430544,0.489535,0.45972,0.430893,...,0.534282,0.464686,0.644914,0.495736,0.469295,0.371949,0.429018,0.166667,0.909091,0.0
3,2014-11-12,0.418884,0.415003,0.420268,0.418080,USD/INR,0.430544,0.493794,0.45972,0.430893,...,0.484180,0.464686,0.644914,0.495489,0.469387,0.371949,0.429018,0.333333,0.909091,0.0
4,2014-11-13,0.417922,0.415431,0.420268,0.419274,USD/INR,0.430544,0.489704,0.45972,0.430893,...,0.516613,0.464686,0.644914,0.496125,0.469614,0.371949,0.429018,0.500000,0.909091,0.0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Convert Date column to datetime (if necessary)
data['Date'] = pd.to_datetime(data['Date'])

# List of unique currency pairs
currency_pairs = data['Currency Pair'].unique()

# Function to create sequences
def create_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1])  # Use all features except the target
        y.append(data[i+seq_length, -1])     # The target variable
    return np.array(X), np.array(y)

# Dictionary to store evaluation metrics for each currency pair
results = {}

# Hyperparameters
SEQ_LENGTH = 30  # Number of time steps in each sequence
EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# Loop over each currency pair
for pair in currency_pairs:
    print(f"Training model for currency pair: {pair}")
    
    # Filter data for the current currency pair
    pair_data = data[data['Currency Pair'] == pair]
    
    # Sort by date to ensure proper time series order
    pair_data = pair_data.sort_values('Date')
    
    # Select features and target (Closing price is the target)
    features = pair_data[['Open_price', 'Day_high', 'Day_low', 'RSI', 'MACD', 'Trend_Open_price', 'Seasonal_Open_price', 'Residual_Open_price', 'BB_Upper', 'BB_Lower']]
    target = pair_data['Closing_price']
    
    # Combine features and target for sequence creation
    data_combined = pd.concat([features, target], axis=1).values
    
    # Split data into train and test sets (80-20 split)
    split_idx = int(len(data_combined) * 0.8)
    train_data = data_combined[:split_idx]
    test_data = data_combined[split_idx:]

    # Create sequences
    X_train, y_train = create_sequences(train_data, SEQ_LENGTH)
    X_test, y_test = create_sequences(test_data, SEQ_LENGTH)
    
    # Build LSTM model
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(SEQ_LENGTH, X_train.shape[2])),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error')
    
    # Train the model
    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), verbose=1)
    
    # Make predictions
    y_pred_test = model.predict(X_test)
    
    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test, y_pred_test)
    mape = mean_absolute_percentage_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)
    
    # Store the results for this currency pair
#     results[pair] = {
#         'MAE': mae,
#         'MAPE': mape,
#         'R^2': r2
#     }
    
#     print(f"Results for {pair} - MAE: {mae}, MAPE: {mape}, R^2: {r2}")

# # Display final results for each currency pair
# print("\nFinal Results for Each Currency Pair:")
# for pair, metrics in results.items():
#     print(f"{pair} - MAE: {metrics['MAE']}, MAPE: {metrics['MAPE']}, R^2: {metrics['R^2']}")
    
 
    # Store the results and predictions for this currency pair
    results[pair] = {
        'MAE': mae,
        'MAPE': mape,
        'R^2': r2,
        'y_true': y_test,       # True values for the test set
        'y_pred': y_pred_test    # Predicted values for the test set
    }
    
print(f"Results for {pair} - MAE: {mae}, MAPE: {mape}, R^2: {r2}")

# Display final results and predictions for each currency pair
print("\nFinal Results for Each Currency Pair:")
for pair, metrics in results.items():
    print(f"{pair} - MAE: {metrics['MAE']}, MAPE: {metrics['MAPE']}, R^2: {metrics['R^2']}")
    print(f"True values (sample): {metrics['y_true'][:5].flatten()}")
    print(f"Predicted values (sample): {metrics['y_pred'][:5].flatten()}\n")


Training model for currency pair: USD/INR
Epoch 1/50


C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0116 - val_loss: 1.1873e-04
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4988e-04 - val_loss: 6.2684e-05
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.6597e-05 - val_loss: 2.7931e-05
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.3152e-05 - val_loss: 1.8481e-05
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.9425e-05 - val_loss: 1.6300e-05
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7245e-05 - val_loss: 1.6500e-05
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5852e-05 - val_loss: 1.6047e-05
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5734e-05 - val_loss: 1.9894e-05
Epoch 9/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.6841e-05 - val_loss: 1.3665e-05
Epoch 10/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4441e-05 - val_loss: 1.3146e-05
Epoch 11/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.3096e-05 - val_loss: 1.0999e-05
E

C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.9925e-04 - val_loss: 9.5056e-06
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.1616e-06 - val_loss: 1.1270e-06
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 9.2043e-07 - val_loss: 6.8124e-07
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.4370e-07 - val_loss: 5.1765e-07
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.7419e-07 - val_loss: 1.0164e-06
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6.0701e-07 - val_loss: 3.5158e-07
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.5958e-07 - val_loss: 3.6809e-07
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.7536e-07 - val_loss: 5.1814e-07
Epoch 9/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.8261e-07 - val_loss: 2.7519e-07
Epoch 10/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.7787e-07 - val_loss: 5.8151e-07
Epoch 11/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.2979e-07 - val_loss: 2.4714e-

...

65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1687 - val_loss: 1.8293e-04
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.6714e-04 - val_loss: 1.0368e-04
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7.9620e-05 - val_loss: 5.5782e-05
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.2598e-05 - val_loss: 4.2465e-05
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.7076e-05 - val_loss: 3.8495e-05
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.2534e-05 - val_loss: 3.6901e-05
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.1325e-05 - val_loss: 5.6215e-05
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.2767e-05 - val_loss: 5.4880e-05
Epoch 9/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.4071e-05 - val_loss: 3.8477e-05
Epoch 10/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.3717e-05 - val_loss: 3.7511e-05
Epoch 11/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.1841e-05 - val_loss: 3.8558e-05
E

C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 1.2565e-04 - val_loss: 9.7685e-06
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8.1359e-06 - val_loss: 4.2205e-06
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.2777e-06 - val_loss: 2.6573e-06
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.3215e-06 - val_loss: 5.4379e-06
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.1551e-06 - val_loss: 4.0075e-06
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7020e-06 - val_loss: 6.7421e-07
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6.7541e-07 - val_loss: 5.8672e-07
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.5342e-07 - val_loss: 8.0267e-07
Epoch 9/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.9560e-07 - val_loss: 6.4171e-07
Epoch 10/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.4416e-07 - val_loss: 5.8479e-07
Epoch 11/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.9406e-07 - val_loss: 2.4029e-

C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0950 - val_loss: 5.0602e-04
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.1842e-04 - val_loss: 1.4251e-04
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.6533e-04 - val_loss: 1.2960e-04
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5975e-04 - val_loss: 1.4280e-04
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5929e-04 - val_loss: 9.4712e-05
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5430e-04 - val_loss: 1.2492e-04
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5628e-04 - val_loss: 1.2789e-04
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4547e-04 - val_loss: 8.9569e-05
Epoch 9/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.3247e-04 - val_loss: 7.2110e-05
Epoch 10/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4037e-04 - val_loss: 7.7424e-05
Epoch 11/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.2602e-04 - val_loss: 6.3904e-05
E